In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go

def plot_calibration(fileName):
    # Load data from Excel
    df = pd.read_excel(fileName, sheet_name="Results")
    
    # Independent variable (Conductivity)
    x1 = df["Conductivity_A_(mS)"]
    x2 = df["Conductivity_B_(mS)"]
    x3 = df["Conductivity_C_(mS)"]
    
    # Dependent variable (Concentration)
    y_from_excel = df["Concentration_(g/L)"]
    
    # Concatenate x values and replicate y values
    x_data = np.concatenate([x1, x2, x3])  # Conductivity
    y_data = np.tile(y_from_excel, 3)  # Concentration

    # Reshape x_data to 2D array for OLS (Zero Intercept)
    X = x_data[:, np.newaxis]  # No intercept

    # Fit the OLS model (forcing zero intercept)
    model = sm.OLS(y_data, X).fit()

    # Confidence intervals
    conf_intervals = model.conf_int()
    
    # Get prediction results
    pred = model.get_prediction(X)
    pred_summary = pred.summary_frame(alpha=0.05)  # 95% CI
    
    # Extract confidence intervals
    ci_lower = pred_summary['mean_ci_lower']
    ci_upper = pred_summary['mean_ci_upper']

    # Create plotly figure
    fig = go.Figure()

    # Plot the three data runs
    fig.add_trace(go.Scatter(x=x1, y=y_from_excel, mode="markers", marker=dict(symbol="x", color="red"), name="Run 1"))
    fig.add_trace(go.Scatter(x=x2, y=y_from_excel, mode="markers", marker=dict(symbol="star", color="blue"), name="Run 2"))
    fig.add_trace(go.Scatter(x=x3, y=y_from_excel, mode="markers", marker=dict(symbol="cross", color="green"), name="Run 3"))

    # Plot the regression model
    slope = model.params[0]  # Only one parameter since intercept is zero
    equation_text = f"y = {slope:.4f}x"  # Equation without intercept

    fig.add_trace(go.Scatter(
        x=x_data, 
        y=model.fittedvalues, 
        mode='lines', 
        name='OLS Model',
        line=dict(color="black"),
        error_y=dict(
            type='data',
            symmetric=False,
            array=ci_upper - model.fittedvalues,  # Upper error bound
            arrayminus=model.fittedvalues - ci_lower,  # Lower error bound
            visible=True,
            color="black"
        )
    ))

    # Add equation annotation
    fig.add_annotation(
        x=x_data.max() - 0.5,
        y=slope * (x_data.max() - 0.5),
        text=f"<b>{equation_text}</b>",
        font=dict(size=14, color="black"),
        align="right",
        xanchor="right",
        yanchor="bottom",
        bgcolor="white"
    )

    # Update layout to swap axes
    fig.update_layout(
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor="gray",
            title="Conductivity (mS)",  # X-axis is now Conductivity
            title_font=dict(size=18),
            tickfont=dict(size=14),
            linewidth=2,
            linecolor="black",
            mirror=True
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor="gray",
            title="Concentration (g/L)",  # Y-axis is now Concentration
            title_font=dict(size=18),
            tickfont=dict(size=14),
            linewidth=2,
            linecolor="black",
            mirror=True
        ),
        title=f"Calibration Curve - {fileName}",
        width=600,
        height=600,
        font=dict(family="Arial", size=14),
        plot_bgcolor="white",
        margin=dict(l=60, r=30, t=30, b=60)
    )

    fig.show()

# Run the function for both files
plot_calibration("RO_Week_1_Data.xlsx")
plot_calibration("RO_Week_2_Data.xlsx")
